## 5. GroupBy a základní transformace

1. Vypočítejte průměrnou maximální teplotu pro každé letiště: `df.groupby('NAME')['TMAX'].mean()`.
2. Agregujte data podle roku a letiště: `df.groupby(['year', 'NAME']).agg({'TMAX': 'mean', 'TMIN': 'mean', 'PRCP': 'sum'})`.
3. Použijte `.transform()` pro výpočet odchylky od průměru letiště: `df['tmax_odchylka'] = df.groupby('NAME')['TMAX'].transform(lambda x: x - x.mean())`.
4. Vytvořte pivot tabulku průměrných maximálních teplot: `pd.pivot_table(df, values='TMAX', index='year', columns='NAME', aggfunc='mean')`.
5. Vytvořte měsíční průměry: `df.groupby(['NAME', 'month'])['TMAX'].mean().unstack()`.
6. Použijte `.rolling(window=7)` na skupinách pro 7denní klouzavé průměry teplot pro každé letiště.
7. Vypočítejte kumulativní srážky v rámci každého roku: `df.sort_values('DATE').groupby(['NAME', 'year'])['PRCP'].cumsum()`.
8. Aplikujte `.rank()` na maximální teploty v rámci každého měsíce pro identifikaci nejteplejších dnů.
9. Použijte `.expanding()` pro výpočet postupně se zvyšujícího průměru teplot od začátku roku.
10. Vytvořte agregaci s více funkcemi: `df.groupby('season').agg({'TMAX': ['mean', 'max', 'min'], 'PRCP': ['sum', 'count']})`.
11. Vypočítejte procentuální podíl srážek v každém měsíci vzhledem k ročnímu úhrnu: `df.groupby(['NAME', 'year', 'month'])['PRCP'].sum().groupby(['NAME', 'year']).transform(lambda x: x / x.sum() * 100)`.
12. Transponujte pivot tabulku z bodu 4 a analyzujte rozdíly mezi letišti.# Cvičení: Analýza meteorologických dat českých letišť

## Scénář
Pracujete jako datový analytik pro řízení letového provozu. Dostali jste historická data o počasí z českých letišť (Ruzyně, Mošnov, Tuřany) za období 2006-2020. Vaším úkolem je analyzovat počasí na těchto letištích pro lepší plánování provozu.

## Popis dat
- STATION: Kód stanice
- NAME: Název letiště (RUZYNE, MOSNOV, TURANY)
- DATE: Datum měření (YYYY-MM-DD)
- PRCP: Srážky v mm
- SNWD: Výška sněhové pokrývky v mm
- TAVG: Průměrná denní teplota v °C (často chybí, nepoužívat)
- TMAX: Maximální denní teplota v °C
- TMIN: Minimální denní teplota v °C

---

## 1. Vytváření Series a DataFrame + Základní průzkum

1. Načtěte soubor 'pydata_data.csv' pomocí `pd.read_csv('pydata_data.csv', sep=';')` a uložte do DataFrame `df`.
2. Zavolejte `.info()` pro zobrazení informací o datasetu včetně datových typů a počtu non-null hodnot.
3. Zobrazte `.shape` a `.columns` datasetu.
4. Použijte `.head(10)` a `.tail(10)` pro kontrolu prvních a posledních řádků.
5. Zjistěte všechny unikátní stanice pomocí `df['STATION'].unique()` a jejich počet pomocí `.nunique()`.
6. Vytvořte Series `temperatures` ze sloupce 'TAVG' a nastavte `.name` na 'Průměrné teploty'.
7. Použijte `.describe()` na celý DataFrame a interpretujte výsledky.
8. Zkontrolujte datové typy všech sloupců pomocí `.dtypes` - všimněte si, že DATE je string.

---

## 2. Filtrování, přistupování na hodnoty a práce s NaN

1. Převeďte sloupec 'DATE' na datetime pomocí `pd.to_datetime(df['DATE'])` a uložte zpět do DataFrame.
2. Vyfiltrujte pouze data z letišť RUZYNE, MOSNOV a TURANY pomocí `df[df['NAME'].isin(['RUZYNE', 'MOSNOV', 'TURANY'])]`.
3. Spočítejte počet chybějících hodnot v každém sloupci pomocí `.isna().sum()`.
4. Vytvořte DataFrame `df_ruzyne` obsahující pouze data z Ruzyně pomocí `df[df['NAME'] == 'RUZYNE']`.
5. Najděte všechny dny s maximální teplotou nad 30°C pomocí `df[df['TMAX'] > 30]`.
6. Vytvořte DataFrame `df_winter` s daty pouze pro zimní měsíce (12, 1, 2) pomocí `df[df['DATE'].dt.month.isin([12, 1, 2])]`.
7. Použijte `.between()` pro nalezení dnů s maximální teplotou mezi 20 a 25°C: `df[df['TMAX'].between(20, 25)]`.
8. Najděte 10 dnů s nejvyššími srážkami pomocí `.nlargest(10, 'PRCP')`.
9. Vyfiltrujte dny se sněhovou pokrývkou pomocí `df[(df['SNWD'] > 0) & (df['SNWD'].notna())]`.
10. Použijte `.loc[]` pro výběr dat z roku 2020 pro Mošnov: `df.loc[(df['DATE'].dt.year == 2020) & (df['NAME'] == 'MOSNOV')]`.
11. Vytvořte náhodný vzorek 1000 řádků pomocí `.sample(n=1000, random_state=42)`.
12. Odstraňte duplicitní záznamy (stejná stanice a datum) pomocí `.drop_duplicates(subset=['STATION', 'DATE'])`.

---

## 3. Přidávání nových sloupců, řádků, odstraňování sloupců

1. Přidejte sloupec 'year' extrahováním roku z DATE: `df['year'] = df['DATE'].dt.year`.
2. Vytvořte sloupec 'month' pomocí `df['month'] = df['DATE'].dt.month`.
3. Přidejte sloupec 'season' pomocí funkce mapující měsíce na roční období: `df['season'] = df['month'].map({12: 'Zima', 1: 'Zima', 2: 'Zima', 3: 'Jaro', 4: 'Jaro', 5: 'Jaro', 6: 'Léto', 7: 'Léto', 8: 'Léto', 9: 'Podzim', 10: 'Podzim', 11: 'Podzim'})`.
4. Vypočítejte teplotní rozpětí: `df['temp_range'] = df['TMAX'] - df['TMIN']`.
5. Pomocí `.assign()` přidejte najednou: 'is_freezing' (TMIN < 0), 'is_hot_day' (TMAX > 25), 'has_precipitation' (PRCP > 0).
6. Použijte `.insert(3, 'day_of_year', df['DATE'].dt.dayofyear)` pro vložení sloupce s pořadím dne v roce.
7. Vytvořte sloupec 'week_number' pomocí `df['week_number'] = df['DATE'].dt.isocalendar().week`.
8. Přidejte sloupec 'decade' pomocí `df['decade'] = (df['year'] // 10) * 10`.
9. Odstraňte sloupec 'TAVG' (obsahuje mnoho NaN) pomocí `.drop(columns=['TAVG'])`.
10. Přejmenujte sloupce na české názvy pomocí `.rename(columns={'PRCP': 'srazky', 'TMAX': 'max_teplota', 'TMIN': 'min_teplota', 'SNWD': 'snih'})`.

---

## 4. Iterace

1. Pomocí `.iterrows()` najděte a vypište všechny dny s teplotou pod -15°C včetně data a letiště.
2. Použijte `.itertuples()` pro výpočet celkových ročních srážek pro každé letiště a rok.
3. Iterujte přes tři letiště (RUZYNE, MOSNOV, TURANY) a pro každé najděte nejteplejší zaznamenaný den.
4. Pomocí `.groupby('NAME').groups` a iterace spočítejte průměrnou maximální teplotu v létě pro každé letiště.
5. Vytvořte dictionary `letiste_extrem = {}` obsahující pro každé letiště nejvyšší a nejnižší teplotu.
6. Najděte nejdelší období bez srážek (po sobě jdoucí dny) pro Ruzyni pomocí iterace.
7. Pro každý měsíc spočítejte, kolik dnů mělo teplotu pod bodem mrazu na všech letištích.

---

## 5. GroupBy a základní transformace

1. Vypočítejte průměrnou teplotu pro každou stanici: `df.groupby('NAME')['TAVG'].mean()`.
2. Agregujte data podle roku a stanice: `df.groupby(['year', 'NAME']).agg({'TAVG': 'mean', 'PRCP': 'sum', 'TMAX': 'max', 'TMIN': 'min'})`.
3. Použijte `.transform()` pro standardizaci teplot v rámci každé stanice: `df.groupby('NAME')['TAVG'].transform(lambda x: (x - x.mean()) / x.std())`.
4. Vytvořte pivot tabulku průměrných teplot: `pd.pivot_table(df, values='TAVG', index='year', columns='NAME', aggfunc='mean')`.
5. Vytvořte měsíční klimatologii: `df.groupby(['NAME', 'month'])['TAVG'].mean().unstack()`.
6. Použijte `.rolling(window=7)` na skupinách pro 7denní klouzavé průměry teplot.
7. Vypočítejte kumulativní srážky v rámci každého roku: `df.groupby(['NAME', 'year'])['PRCP'].cumsum()`.
8. Aplikujte `.rank()` na teploty v rámci každého měsíce pro identifikaci nejteplejších dnů.
9. Použijte `.expanding()` pro výpočet historického maxima teploty do daného dne.
10. Vytvořte agregaci s vlastními funkcemi: `df.groupby('season').agg({'TAVG': ['mean', 'std', lambda x: x.quantile(0.9)]})`.
11. Normalizujte srážky na roční úroveň pomocí `.groupby(['NAME', 'year'])['PRCP'].transform(lambda x: x / x.sum() * 100)`.
12. Transponujte pivot tabulku z bodu 4 a analyzujte rozdíly mezi městy.

---

## 6. MultiIndex

1. Vytvořte MultiIndex z roku, měsíce a názvu stanice: `df.set_index(['year', 'month', 'NAME'])`.
2. Na MultiIndex DataFrame vyberte všechna data pro rok 2019: `.loc[2019]`.
3. Vyberte letní měsíce všech let pro Prahu: `.loc[(slice(None), [6,7,8], 'PRAHA KLEMENTINUM'), :]`.
4. Použijte `.xs()` pro výběr července napříč všemi roky a stanicemi: `.xs(7, level='month')`.
5. Prohoďte úrovně indexu pomocí `.swaplevel(0, 2)` a seřaďte `.sort_index()`.
6. Vytvořte agregaci s MultiIndexem pro rok/měsíc/stanici a aplikujte `.unstack(level='NAME')`.
7. Použijte `.droplevel()` pro odstranění úrovně měsíce z indexu.
8. Vytvořte MultiIndex pomocí `pd.MultiIndex.from_product()` pro všechny kombinace let a měsíců.
9. Resetujte pouze jednu úroveň MultiIndexu pomocí `.reset_index(level='month')`.
10. Vytvořte hierarchické seskupení podle dekády, roku a sezóny.

---

## 7. Základní vizualizace

1. Vytvořte figure s 2x3 subploty: `fig, axes = plt.subplots(2, 3, figsize=(18, 12))`.
2. Na `axes[0,0]` nakreslete časovou řadu maximálních teplot pro Ruzyni: `df[df['NAME']=='RUZYNE'].plot(x='DATE', y='TMAX', ax=axes[0,0], alpha=0.5, color='red')`.
3. Na `axes[0,1]` vytvořte boxplot teplot podle měsíce pro Mošnov: `df[df['NAME']=='MOSNOV'].boxplot(column='TMAX', by='month', ax=axes[0,1])`.
4. Na `axes[0,2]` nakreslete histogram teplotního rozpětí pro Tuřany: `df[df['NAME']=='TURANY']['temp_range'].hist(bins=50, ax=axes[0,2], edgecolor='black', color='green')`.
5. Na `axes[1,0]` vytvořte scatter plot TMIN vs TMAX pro všechna letiště s různými barvami: `for name, group in df.groupby('NAME'): group.plot.scatter(x='TMIN', y='TMAX', ax=axes[1,0], label=name, alpha=0.3)`.
6. Na `axes[1,1]` nakreslete sloupcový graf průměrných ročních srážek podle letiště: `df.groupby(['NAME', 'year'])['PRCP'].sum().groupby('NAME').mean().plot(kind='bar', ax=axes[1,1], color=['blue', 'orange', 'green'])`.
7. Na `axes[1,2]` vytvořte line plot srovnání měsíčních průměrů teplot mezi letišti: `df.groupby(['NAME', 'month'])['TMAX'].mean().unstack(level=0).plot(ax=axes[1,2], marker='o')`.
8. Pro každý subplot nastavte title: `axes[0,0].set_title('Vývoj maximálních teplot - Ruzyně')`, atd.
9. Nastavte popisky os: `axes[0,0].set_xlabel('Datum')`, `axes[0,0].set_ylabel('Teplota [°C]')`.
10. Přidejte mřížku na všechny grafy: `for ax in axes.flat: ax.grid(True, alpha=0.3, linestyle='--')`.
11. Nastavte limity os pro scatter plot: `axes[1,0].set_xlim(-25, 15)`, `axes[1,0].set_ylim(-15, 40)`.
12. Přidejte legendu: `axes[1,0].legend(title='Letiště', loc='upper left')`, `axes[1,2].legend(title='Letiště', bbox_to_anchor=(1.05, 1))`.
13. Použijte různé styly čar pro line plot: `axes[1,2].lines[0].set_linestyle('-')`, `axes[1,2].lines[1].set_linestyle('--')`, `axes[1,2].lines[2].set_linestyle(':')`.
14. Naformátujte osu x pro časové řady: `import matplotlib.dates as mdates; axes[0,0].xaxis.set_major_locator(mdates.YearLocator(2)); axes[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))`.
15. Použijte `plt.tight_layout()` a uložte: `plt.tight_layout(); plt.savefig('letiste_analyza.png', dpi=300, bbox_inches='tight')`.

---

## 8. Kategorizace (cut, qcut)

1. Kategorizujte teploty do intervalů: `pd.cut(df['TAVG'], bins=[-20, -10, 0, 10, 20, 30, 40], labels=['Mrazivá', 'Velmi chladná', 'Chladná', 'Mírná', 'Teplá', 'Horká'])`.
2. Vytvořte kvintily pro srážky: `pd.qcut(df['PRCP'], q=5, labels=['Velmi suché', 'Suché', 'Normální', 'Vlhké', 'Velmi vlhké'])`.
3. Kategorizujte sněhovou pokrývku: `pd.cut(df['SNWD'], bins=[0, 10, 50, 100, 1000], labels=['Žádný', 'Mírný', 'Střední', 'Vysoký'], include_lowest=True)`.
4. Vytvořte teplotní percentily: `pd.qcut(df['TAVG'], q=[0, 0.1, 0.25, 0.75, 0.9, 1.0], labels=['Extrémně chladná', 'Chladná', 'Normální', 'Teplá', 'Extrémně teplá'])`.
5. Kategorizujte dny podle teplotního rozpětí: `pd.cut(df['temp_range'], bins=5)`.
6. Pro každou kategorii spočítejte četnosti: `.value_counts()` a relativní četnosti: `.value_counts(normalize=True)`.
7. Vytvořte kontingenční tabulku kategorií teplot vs. kategorií srážek: `pd.crosstab()`.
8. Analyzujte rozložení kategorií podle ročních období.
9. Použijte `.cat.codes` pro získání numerických hodnot kategorií.
10. Přeuspořádejte kategorie podle vlastního pořadí pomocí `.cat.reorder_categories()`.

---

## 9. Korelace

1. Vypočítejte korelační matici mezi teplotami a srážkami: `df[['TMAX', 'TMIN', 'PRCP', 'temp_range']].corr()`.
2. Pro každé letiště zvlášť vypočítejte korelaci mezi maximální teplotou a srážkami: `df.groupby('NAME')[['TMAX', 'PRCP']].corr()`.
3. Vypočítejte Spearmanovu korelaci: `df[['TMAX', 'PRCP']].corr(method='spearman')`.
4. Zjistěte korelaci mezi minimální a maximální teplotou pouze pro zimní měsíce: `df[df['season']=='Zima'][['TMIN', 'TMAX']].corr()`.
5. Vypočítejte korelaci mezi teplotou a pořadím dne v roce (vliv ročních období): `df[['TMAX', 'day_of_year']].corr()`.
6. Pro každý rok vypočítejte korelaci mezi teplotou a srážkami na Ruzyni: `df[df['NAME']=='RUZYNE'].groupby('year')[['TMAX', 'PRCP']].corr()`.
7. Zjistěte, které letiště má nejsilnější korelaci mezi minimální teplotou a sněhovou pokrývkou.
8. Vypočítejte korelaci mezi teplotním rozpětím a srážkami: `df[['temp_range', 'PRCP']].corr()`.
9. Identifikujte měsíce s nejvyšší korelací mezi letišti (podobné počasí).
10. Vizualizujte korelační matici mezi letišti pro maximální teploty.

---

## 10. Pokročilé transformace (stack, unstack, melt, explode)

1. Vytvořte wide-format tabulku s městy jako sloupci a daty jako řádky, pak použijte `.stack()`.
2. Na výsledek z bodu 1 aplikujte `.unstack(level='NAME')`.
3. Použijte `pd.melt()` s parametry: `id_vars=['DATE', 'NAME']`, `value_vars=['TMAX', 'TMIN', 'TAVG']`, `var_name='temp_type'`, `value_name='temperature'`.
4. Vytvořte pivot tabulku měsíčních průměrů a použijte `.stack()` pro long format.
5. Aplikujte `.unstack()` na MultiIndex DataFrame pro vytvoření wide formátu podle měsíců.
6. Použijte `pd.crosstab()` s normalizací pro analýzu výskytu extrémních teplot podle města a roku.
7. Vytvořte hierarchickou strukturu dat pomocí kombinace `.stack()` a `.unstack()`.
8. Transformujte data pro time series analýzu pomocí `.pivot()` a `.melt()`.
9. Vytvořte multi-level columns pomocí `.unstack()` na skupinových agregacích.
10. Použijte `.droplevel()` a `.swaplevel()` pro úpravu výsledné struktury.

---

## 11. Spojování DataFrame (concat, merge, join, combine)

1. Rozdělte data podle letiště a spojte zpět: `df_ruz = df[df['NAME']=='RUZYNE']`, `df_mos = df[df['NAME']=='MOSNOV']`, `df_tur = df[df['NAME']=='TURANY']`, pak `pd.concat([df_ruz, df_mos, df_tur], keys=['Ruzyně', 'Mošnov', 'Tuřany'])`.
2. Vytvořte DataFrame s měsíčními průměry teplot a mergujte s denními daty: `monthly_avg = df.groupby(['NAME', 'year', 'month'])['TMAX'].mean().reset_index()`, pak `df.merge(monthly_avg, on=['NAME', 'year', 'month'], suffixes=('', '_monthly_avg'))`.
3. Vytvořte kalendář všech dnů pomocí `pd.date_range('2006-01-01', '2020-12-31', freq='D')` a left join s daty pro identifikaci chybějících měření: `calendar = pd.DataFrame({'DATE': pd.date_range('2006-01-01', '2020-12-31', freq='D')})`, pak `calendar.merge(df[df['NAME']=='RUZYNE'], on='DATE', how='left')`.
4. Vytvořte DataFrame s českými svátky (alespoň 5 významných dat) a spojte s weather daty: `svatky = pd.DataFrame({'DATE': pd.to_datetime(['2020-01-01', '2020-05-01', '2020-05-08', '2020-09-28', '2020-12-24']), 'svatek': ['Nový rok', 'Svátek práce', 'Den vítězství', 'Den české státnosti', 'Štědrý den']})`, pak merge.
5. Mergujte data různých letišť podle data pro srovnání: `ruzyne = df[df['NAME']=='RUZYNE'].set_index('DATE')`, `mosnov = df[df['NAME']=='MOSNOV'].set_index('DATE')`, pak `ruzyne.merge(mosnov, left_index=True, right_index=True, suffixes=('_ruz', '_mos'))`.
6. Vytvořte DataFrame s ročními statistikami a joinujte zpět: `yearly_stats = df.groupby(['NAME', 'year']).agg({'TMAX': 'mean', 'PRCP': 'sum'}).add_suffix('_yearly')`.
7. Použijte `.combine_first()` pro doplnění chybějících hodnot z jiného letiště (např. když chybí data z Ruzyně, doplňte z Mošnova).
8. Demonstrujte všechny typy joins při spojování dat Ruzyně a Tuřan pro rok 2020: inner (pouze společné dny), left (všechny dny Ruzyně), right (všechny dny Tuřan), outer (všechny dny obou).
9. Vytvořte DataFrame s průměry za dekádu a připojte k původním datům.
10. Použijte `pd.concat()` s `axis=1` pro horizontální spojení teplotních a srážkových analýz.
11. Vytvořte hierarchické spojení dat z různých let pomocí MultiIndex.
12. Porovnejte data před a po opravě chyb pomocí `.compare()`.

---

## 12. Time Intelligence (pro denní data)

1. Nastavte DATE jako DatetimeIndex: `df.set_index('DATE', inplace=True)`.
2. Použijte `.resample('W')` pro týdenní agregaci s průměrnými teplotami a součtem srážek.
3. Vytvořte měsíční agregace: `.resample('M').agg({'TAVG': 'mean', 'PRCP': 'sum', 'TMAX': 'max', 'TMIN': 'min'})`.
4. Aplikujte `.resample('Q')` pro čtvrtletní statistiky a `.resample('Y')` pro roční.
5. Použijte `.rolling('30D')` pro 30denní klouzavé průměry teplot.
6. Vytvořte meziroční srovnání pomocí `.shift(365)` a vypočítejte změny.
7. Použijte `.expanding()` pro výpočet rekordních teplot do daného data.
8. Aplikujte různé window funkce: `.rolling('14D').mean()`, `.rolling('7D').std()`.
9. Vytvořte custom offset pro pracovní dny: `pd.offsets.BusinessDay(5)`.
10. Použijte `.asfreq('D')` s `method='ffill'` pro doplnění chybějících dnů.
11. Vytvořte sezónní dekompozici pomocí 365denního rolling mean.
12. Aplikujte `.shift()` s různými periodami pro lag analysis.
13. Použijte `pd.Grouper(freq='M')` v kombinaci s groupby pro měsíční agregace zachovávající DatetimeIndex.
14. Vytvořte `PeriodIndex` s měsíční frekvencí a demonstrujte převody mezi Period a Timestamp.
15. Vypočítejte klouzavé korelace mezi teplotou a srážkami s měnícím se oknem.

---

## Závěrečný projekt

Vytvořte komplexní analýzu počasí na českých letištích obsahující:

1. **Srovnání letišť**: Které letiště je nejteplejší/nejchladnější? Kde prší nejvíce?
2. **Sezónní analýza**: Vytvořte přehled typického počasí pro každé roční období na každém letišti (průměrné teploty, srážky).
3. **Extrémní události**: Identifikujte:
   - Nejteplejší dny (TMAX > 35°C)
   - Nejchladnější dny (TMIN < -15°C)
   - Dny s nejvyššími srážkami (PRCP > 50mm)
   - Nejdelší období bez deště (20+ dní bez srážek)
4. **Trendy v čase**: Mění se průměrné teploty nebo srážky v průběhu let 2006-2020?
5. **Provozní analýza pro letiště**:
   - Ve kterých měsících je nejvíce dnů se sněhem (problém pro provoz)?
   - Kdy je největší teplotní rozpětí během dne (namrzání)?
   - Které měsíce mají nejstabilnější počasí?
6. **Dashboard**: Vytvořte přehlednou vizualizaci s minimálně 8 grafy zobrazujícími:
   - Vývoj teplot v čase
   - Srovnání letišť
   - Sezónní vzory
   - Rozložení srážek
   - Extrémní hodnoty
7. **Doporučení**: Na základě analýzy navrhněte:
   - Nejlepší měsíce pro plánovanou údržbu (stabilní počasí, málo extrémů)
   - Období vyžadující zvýšenou připravenost (sníh, mráz, vysoké teploty)